# Azure Text Translator

Este notebook apresenta um exemplo em Python de acionamento da API Azure Text Translator.

* [API no Hub de Modelos](https://apid-portal.petrobras.com.br/api-details#api=azure-text-translator)
* [Documentação oficial na Azure](https://learn.microsoft.com/en-us/azure/ai-services/translator/reference/v3-0-reference)

In [1]:
import os
import requests
import json
from configparser import ConfigParser, ExtendedInterpolation

# Configura o certificado Petrobras para a lib requests
PTB_CERT_PATH = os.path.join(os.path.abspath(''), '../petrobras-ca-root.pem')
os.environ['REQUESTS_CA_BUNDLE'] = PTB_CERT_PATH

session = requests.Session()

# Obtém as credenciais do arquivo de config
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../config.ini', 'UTF-8')

# URL base da API no Hub de Modelos (APIM)
BASE_URL = 'https://apid.petrobras.com.br/ia/texto/v1/azure-text-translator'

# Headers enviado nas requisições
default_headers = {
    "Content-Type": "application/json",
    "apikey": config['HUB']['apikey_modelos_texto'],
}


# Método utilitário para fazer uma requisição e retornar o body em JSON
# Em caso de status diferente de sucesso (200), lança uma exceção
def get_json_checking_error(path, method = 'POST', json_input = {}, verify_certificate=True):
    url = f'{BASE_URL}{path}'
    print(f'{method} {url}')
    response = session.request(method, url, headers=default_headers, verify=verify_certificate, json=json_input)
    print(f'Tempo: {response.elapsed.total_seconds()}s')
    if response.status_code != 200:
        raise RuntimeError(f'Erro no response ({response.status_code}): {response.json()}')
    return response.json()


In [6]:
json_input = [
    {
        'Text': 'A Petrobras é referência internacional na exploração de petróleo em águas profundas, para a qual desenvolveu tecnologia própria, pioneira no mundo, sendo a líder mundial deste setor.'
    },
    {
        'Text': 'O petróleo está associado a grandes estruturas que comunicam a crosta e o manto da Terra, sobretudo nos limites entre placas tectônicas.'
    }
]

traducoes = get_json_checking_error('/translator/text/v3.0/translate?from=pt&to=en', json_input=json_input)

print(json.dumps(traducoes, indent=4))

POST https://apid.petrobras.com.br/ia/texto/v1/azure-text-translator/translator/text/v3.0/translate?from=pt&to=en
Tempo: 0.200203s
[
    {
        "translations": [
            {
                "text": "Petrobras is an international reference in deepwater oil exploration, for which it has developed its own technology, a pioneer in the world, being the world leader in this sector.",
                "to": "en"
            }
        ]
    },
    {
        "translations": [
            {
                "text": "Oil is associated with large structures that communicate the Earth's crust and mantle, especially at the boundaries between tectonic plates.",
                "to": "en"
            }
        ]
    }
]


In [3]:
# Método para gerar exemplos de uso de uma palavra em um contexto

json_input = [
    {
        'Text': 'livro',
        'Translation': 'book'
    }
]

exemplos = get_json_checking_error('/translator/text/v3.0/dictionary/examples?api-version=3.0&from=pt&to=en', json_input=json_input)
print(json.dumps(exemplos, indent=4))

POST https://apid.petrobras.com.br/ia/texto/v1/azure-text-translator/translator/text/v3.0/dictionary/examples?api-version=3.0&from=pt&to=en
Tempo: 0.236689s
[
    {
        "normalizedSource": "livro",
        "normalizedTarget": "book",
        "examples": [
            {
                "sourcePrefix": "Esse ",
                "sourceTerm": "livro",
                "sourceSuffix": " teve muito sucesso.",
                "targetPrefix": "That ",
                "targetTerm": "book",
                "targetSuffix": " was very successful."
            },
            {
                "sourcePrefix": "Este ",
                "sourceTerm": "livro",
                "sourceSuffix": " \u00e9 uma falsifica\u00e7\u00e3o.",
                "targetPrefix": "This ",
                "targetTerm": "book",
                "targetSuffix": " is a forgery."
            },
            {
                "sourcePrefix": "Mas \u00e9 um ",
                "sourceTerm": "livro",
                "sourceSuffix

In [4]:
# Método para detectar o idioma de uma frase

json_input = [
    {
        'Text': 'Durante este período, Petrobras se convirtió en el líder en el comercio de derivados en Brasil.'
    }
]

print(json.dumps(get_json_checking_error('/translator/text/v3.0/detect', json_input=json_input), indent=4))

POST https://apid.petrobras.com.br/ia/texto/v1/azure-text-translator/translator/text/v3.0/detect
Tempo: 0.200554s
[
    {
        "isTranslationSupported": true,
        "isTransliterationSupported": false,
        "language": "es",
        "score": 1.0
    }
]
